In [1]:
import sys
import os
import random
import pandas as pd
import pandas as pd
import numpy as np
import torch
from tqdm import trange

In [2]:
paths = {
    'train_index':'./data/train.csv',
    'test_index':'./data/test.csv',
    'val_index':'./data/valid.csv',
}


In [3]:
train = pd.read_csv(paths['train_index'], dtype=str)
test = pd.read_csv(paths['test_index'], dtype=str)
val = pd.read_csv(paths['val_index'], dtype=str)

train.columns = ['user','top','bottom','negative']
test.columns = ['user','top','bottom','negative']
val.columns = ['user','top','bottom','negative']

total = pd.concat((train, test, val), ignore_index=True)
total

,user,top,bottom,negative
0,2598816,40382691,40382734,27787437
1,2598816,40368970,40369314,46569173
2,2598816,40382363,40663774,37025670
3,2598816,35616253,29951608,15984921
4,2598816,29477635,39399258,9554875
...,...,...,...,...
216765,2361321,41774386,41893909,42332692
216766,2645816,42336635,42138634,4570225
216767,2606685,33835769,31666306,15648586
216768,2638960,37772139,39341584,38943070


In [4]:
user_dict = pd.read_csv("id_user.txt", header=None)
user_dict.columns = ['id']
user_dict.insert(0, 'new_id', range(1, 1 + len(user_dict)))
user_dict
user_dictt = user_dict.set_index('id').T.to_dict('records')[0]
print(user_dictt)

{2598816: 1, 2358846: 2, 2516446: 3, 2266445: 4, 2402247: 5, 2395430: 6, 1527904: 7, 2596447: 8, 1443842: 9, 2603295: 10, 702676: 11, 454345: 12, 2592377: 13, 2339891: 14, 2493929: 15, 799696: 16, 29226: 17, 2553774: 18, 2562091: 19, 73855: 20, 2514223: 21, 2527625: 22, 2386285: 23, 954230: 24, 391430: 25, 2461578: 26, 832880: 27, 2461278: 28, 943840: 29, 2641968: 30, 2392883: 31, 2645454: 32, 2596233: 33, 1482757: 34, 2454445: 35, 160785: 36, 2456363: 37, 2517188: 38, 2528781: 39, 2389260: 40, 2497441: 41, 287951: 42, 150826: 43, 2304118: 44, 2327063: 45, 2017292: 46, 2489799: 47, 2645876: 48, 2642809: 49, 2641760: 50, 2544785: 51, 9276: 52, 2466048: 53, 2604875: 54, 1805373: 55, 2507560: 56, 650119: 57, 1675484: 58, 2640936: 59, 2223005: 60, 2066607: 61, 2600363: 62, 2649374: 63, 790119: 64, 2435425: 65, 864785: 66, 1063668: 67, 1481337: 68, 2394424: 69, 2122761: 70, 2471945: 71, 2524605: 72, 2510562: 73, 651275: 74, 2391692: 75, 2647073: 76, 2444562: 77, 1011044: 78, 2647287: 79, 25

In [5]:
bottom_dict = pd.read_csv("id_bottom.txt", header=None)
bottom_dict.columns = ['id']
bottom_dict.insert(0, 'new_id', range(1, 1 + len(bottom_dict)))
bottom_dict
bottom_dictt = bottom_dict.set_index('id').T.to_dict('records')[0]
print(bottom_dictt)

{40382734: 1, 40369314: 2, 40663774: 3, 29951608: 4, 39399258: 5, 41181999: 6, 37350685: 7, 41236231: 8, 42139220: 9, 26872498: 10, 38115636: 11, 31343682: 12, 39044838: 13, 36933574: 14, 41108556: 15, 36220184: 16, 37649451: 17, 31885640: 18, 38655042: 19, 35331815: 20, 43550713: 21, 15249151: 22, 36444080: 23, 35808883: 24, 38652677: 25, 44885459: 26, 35807939: 27, 36426663: 28, 42272536: 29, 36475908: 30, 41945890: 31, 34018946: 32, 42888816: 33, 38192005: 34, 39676407: 35, 34464456: 36, 37027574: 37, 35570405: 38, 13602941: 39, 34424888: 40, 48127995: 41, 37636125: 42, 34357411: 43, 41681902: 44, 20288050: 45, 41934107: 46, 13589205: 47, 42761433: 48, 33380704: 49, 38829797: 50, 38624619: 51, 24387239: 52, 32618354: 53, 42166328: 54, 38186505: 55, 20448543: 56, 13588917: 57, 37416076: 58, 42259132: 59, 27244178: 60, 43284814: 61, 43313669: 62, 41490752: 63, 13576079: 64, 42070130: 65, 34823821: 66, 13589792: 67, 35153054: 68, 38065833: 69, 28516289: 70, 41893914: 71, 40840044: 72, 

In [6]:
#generate the full item dict(for test_data purpose)
top_dict = pd.read_csv("id_top.txt", header=None)
top_dict.columns = ['id']
bottom_dict = pd.read_csv("id_bottom.txt", header=None)
bottom_dict.columns = ['id']
item_dict = pd.concat([bottom_dict,top_dict])
item_dict.insert(0, 'new_id', range(1, 1 + len(item_dict)))
item_dict
item_dictt = item_dict.set_index('id').T.to_dict('records')[0]
print(item_dictt)

<ipython-input-6-362a95f506cd>:9: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  item_dictt = item_dict.set_index('id').T.to_dict('records')[0]


{40382734: 1, 40369314: 2, 40663774: 3, 29951608: 4, 39399258: 5, 41181999: 6, 37350685: 7, 41236231: 8, 42139220: 9, 26872498: 10, 38115636: 11, 31343682: 12, 39044838: 13, 36933574: 14, 41108556: 15, 36220184: 16, 37649451: 17, 31885640: 18, 38655042: 19, 35331815: 20, 43550713: 21, 15249151: 22, 36444080: 23, 35808883: 24, 38652677: 25, 44885459: 26, 35807939: 27, 36426663: 28, 42272536: 29, 36475908: 30, 41945890: 31, 34018946: 32, 42888816: 33, 38192005: 34, 39676407: 35, 34464456: 36, 37027574: 37, 35570405: 38, 13602941: 39, 34424888: 40, 48127995: 41, 37636125: 42, 34357411: 43, 41681902: 44, 20288050: 45, 41934107: 46, 13589205: 47, 42761433: 48, 33380704: 49, 38829797: 50, 38624619: 51, 24387239: 52, 32618354: 53, 42166328: 54, 38186505: 55, 20448543: 56, 13588917: 57, 37416076: 58, 42259132: 59, 27244178: 60, 43284814: 61, 43313669: 62, 41490752: 63, 13576079: 64, 42070130: 65, 34823821: 66, 13589792: 67, 35153054: 68, 38065833: 69, 28516289: 70, 41893914: 71, 40840044: 72, 

In [7]:
np.save('user_dict.npy', user_dictt)
np.save('item_dict.npy', item_dictt)

In [8]:
# importing the module
import pickle
  
# opening file in write mode (binary)
with open("user_dict.txt", "wb") as file:
  
    # serializing dictionary 
    pickle.dump(user_dictt, file)
    
with open("item_dict.txt", "wb") as file:
  
    # serializing dictionary 
    pickle.dump(item_dictt, file)

In [26]:
#generate new training data
filename = "old/user_bottom_train.txt"
outline = ""
with open(filename, "r") as fn:
    for line in fn.read().splitlines():
        arr = line.split(" ")
        user_id = user_dictt[int(arr[0])]
        bottom_id = bottom_dictt[int(arr[1])]
        new_line = str(user_id) + " " + str(bottom_id)
        #neg_id = item_dict[arr[2]]
        outline+= new_line+'\n'

In [28]:
#save new training data
outputfile = "new/user_bottom_train.txt"
with open(outputfile, "w") as op:
    op.write(outline)

In [40]:
#generate new testing data
filename = "old/user_bottom_test.txt"
outline = ""
with open(filename, "r") as fn:
    for line in fn.read().splitlines():
        arr = line.split(" ")
        user_id = user_dictt[int(arr[0])]
        bottom_id = bottom_dictt[int(arr[1])]
        neg_id = item_dictt[int(arr[2])]
        new_line = str(user_id) + " " + str(bottom_id) + " " + str(neg_id)
        
        outline+= new_line+'\n'

In [41]:
#save new testing data
outputfile = "new/user_bottom_test.txt"
with open(outputfile, "w") as op:
    op.write(outline)

In [50]:
#new form of testing data
filename = "new/user_bottom_test.txt"
outline,user,true,neg = "","","",""
with open(filename, "r") as fn:
    for line in fn.read().splitlines():
        arr = line.split(" ")
        if arr[0]==user: #同一user的情況下
            true = true+' '+arr[1]
            neg = neg +' '+arr[2]
            
        else: #不同user的情況下
            if user!="":
                new_line = user+","+true+","+neg
                outline+= new_line+'\n'
            user = arr[0]
            true = arr[1]
            neg = arr[2]

In [5]:
#remove top items from negative test datas
filename = "new/old/user_bottom_test.txt"
outline = ""
with open(filename, "r") as fn:
    for line in fn.read().splitlines():
        arr = line.split(",")
        user,true,neg = "","",""
        user = arr[0]
            
        for idx, arr1 in enumerate(arr[1].split(" ")):
            if int(arr1)> 43085:
                continue
            else:
                if idx == 0:
                    true = arr1
                else:
                    true = true+' '+arr1
            
        for idx, arr2 in enumerate(arr[2].split(" ")):
            if int(arr2)> 43085:
                continue
            else:
                if idx == 0:
                    neg = arr2
                else:
                    neg = neg +' '+arr2
            
        new_line = user+","+true+","+neg
        outline+= new_line+'\n'


In [6]:
#save new form of testing data
outputfile = "new/new/user_bottom_test.txt"
with open(outputfile, "w") as op:
    op.write(outline)

In [43]:
#generate new embedding data with new ids
filename = "old/top_node_emb.txt"
outline = ""
with open(filename, "r") as fn:
    for line in fn.read().splitlines():
        arr = line.split(",")
        arr[0] = str(item_dictt[int(arr[1])])
        new_line = ','.join(arr)
        outline+= new_line+'\n'

In [44]:
#save new embedding data
outputfile = "new/top_node_emb.txt"
with open(outputfile, "w") as op:
    op.write(outline)

In [9]:
#generate new metapath data with new ids
filename = "old/pmf_outfitmetapath_005.txt"
outline = ""
with open(filename, "r") as fn:
    for line in fn.read().splitlines():
        arr = line.split(" ")
        new_path = ""
        for idx,node in enumerate(arr[0].split("-")):
            if(idx == 0):
                tmp = 'u'+str(user_dictt[int(node.strip('u'))])
            elif(idx == 1):
                tmp = 't'+str(item_dictt[int(node.strip('t'))])
            elif(idx == 2):
                tmp = 'b'+str(item_dictt[int(node.strip('b'))])
            new_path+=tmp+'-'
            
        new_path = new_path.rstrip('-')+" "+str(arr[1])
        outline+= new_path+'\n'

In [10]:
#save new embedding data
outputfile = "new/pmf_outfitmetapath_005.txt"
with open(outputfile, "w") as op:
    op.write(outline)

In [5]:
user_bottom = test.iloc[:,[0,2,3]]
user_bottom
user_bottom.to_csv("user_bottom_test.txt", header=None, index=None, sep=' ')

In [23]:
total['pair']= total['top']+"_"+total['bottom']
total

,user,top,bottom,negative,pair
0,2598816,40382691,40382734,27787437,40382691_40382734
1,2598816,40368970,40369314,46569173,40368970_40369314
2,2598816,40382363,40663774,37025670,40382363_40663774
3,2598816,35616253,29951608,15984921,35616253_29951608
4,2598816,29477635,39399258,9554875,29477635_39399258
...,...,...,...,...,...
216765,2361321,41774386,41893909,42332692,41774386_41893909
216766,2645816,42336635,42138634,4570225,42336635_42138634
216767,2606685,33835769,31666306,15648586,33835769_31666306
216768,2638960,37772139,39341584,38943070,37772139_39341584


In [24]:
user_pair = total.iloc[:,[0,4]]
user_pair.to_csv("user_pair.txt", header=None, index=None, sep=' ')

In [ ]:
#user_top = pd.concat([total['user'],total['top']])
user_top = total.iloc[:,0:2]
user_top.to_csv("user_top.txt", header=None, index=None, sep=' ')

In [ ]:
top_bottom = total.iloc[:,1:3]
top_bottom.to_csv("top_bottom.txt", header=None, index=None, sep=' ')

In [ ]:
id_user = total.iloc[:,0:1]
id_user = id_user.astype(int)
id_user = id_user.drop_duplicates(subset=None, keep='first', inplace=False)
id_user.to_csv("id_user.txt", header=None, index=None, sep=' ')

In [ ]:
id_bottom = total.iloc[:,2:3]
id_bottom = id_bottom.astype(int)
id_bottom = id_bottom.drop_duplicates(subset=None, keep='first', inplace=False)
id_bottom.to_csv("id_bottom.txt", header=None, index=None, sep=' ')

In [ ]:
user_bottom = total.iloc[:,[0,2]]
user_bottom.to_csv("user_bottom.txt", header=None, index=None, sep=' ')

In [ ]:
id_top = total.iloc[:,1:2]
id_top = id_top.astype(int)
id_top = id_top.drop_duplicates(subset=None, keep='first', inplace=False)
id_top.to_csv("id_top.txt", header=None, index=None, sep=' ')

In [ ]:
bottom_top = total.iloc[:,[2,1]]
bottom_top.to_csv("bottom_top.txt", header=None, index=None, sep=' ')

In [ ]:
bottom_user = total.iloc[:,[2,0]]
bottom_user.to_csv("bottom_user.txt", header=None, index=None, sep=' ')

In [ ]:
user_bottom['value'] = 1
user_bottom=user_bottom.drop_duplicates()
user_bottom

<ipython-input-14-22d8f3805c8c>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  user_bottom['value'] = 1


,user,bottom,value
0,2598816,40382734,1
1,2598816,40369314,1
2,2598816,40663774,1
3,2598816,29951608,1
4,2598816,39399258,1
...,...,...,...
216764,694245,4578539,1
216766,2645816,42138634,1
216767,2606685,31666306,1
216768,2638960,39341584,1


In [15]:
user_bot_matrix = user_bottom.pivot(index='user', columns='bottom', values='value')
user_bot_matrix

bottom,10001094,10002250,10002811,10003235,10003243,10003245,10003936,10003938,10004507,10005559,...,9996947,9996949,9996951,9997285,9997301,9997304,9997305,9997308,9997335,9998530
user,,,,,,,,,,,,,,,,,,,,,
1000092,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1000325,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1003407,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1009646,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1009734,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
990875,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
996095,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
996110,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
n_users, n_movies = user_bot_matrix.shape
sparcity = user_bot_matrix.notna().sum().sum() / (n_users * n_movies)
print(f'Sparcity: {sparcity:0.2%}')

Sparcity: 0.11%


In [17]:
class PMFLoss(torch.nn.Module):
    def __init__(self, lam_u=0.3, lam_v=0.3):
        super().__init__()
        self.lam_u = lam_u
        self.lam_v = lam_v
    
    def forward(self, matrix, u_features, v_features):
        predicted = torch.sigmoid(torch.matmul(u_features, v_features.t()))
        
        diff = (matrix - predicted)**2
        prediction_error = torch.sum(diff)

        u_regularization = self.lam_u * torch.sum(u_features.norm(dim=1))
        v_regularization = self.lam_v * torch.sum(v_features.norm(dim=1))
        
        return prediction_error + u_regularization + v_regularization

In [18]:
# Actual training loop now
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

latent_vectors = 30
num_epochs = 1000
#Replacing N/A with 0
user_bot_matrix = user_bot_matrix.fillna(value=0)



In [19]:
# Replacing missing ratings with -1 so we can filter them out later
#user_bot_matrix[rating_matrix.isna()] = -1
user_bot_matrix = torch.from_numpy(user_bot_matrix.values).to(device)
#non_zero_mask = (user_bot_matrix != -1)

user_features = torch.randn(n_users, latent_vectors, requires_grad=True, device=device)
user_features.data.mul_(0.01)
movie_features = torch.randn(n_movies, latent_vectors, requires_grad=True, device=device)
movie_features.data.mul_(0.01)


tensor([[-0.0080, -0.0070,  0.0153,  ..., -0.0100, -0.0034,  0.0053],
        [ 0.0041, -0.0087, -0.0002,  ..., -0.0022,  0.0015, -0.0016],
        [ 0.0035, -0.0024, -0.0074,  ...,  0.0058,  0.0039,  0.0059],
        ...,
        [-0.0002,  0.0111,  0.0238,  ..., -0.0173, -0.0052,  0.0062],
        [ 0.0025, -0.0074,  0.0083,  ...,  0.0005,  0.0109, -0.0022],
        [ 0.0003, -0.0049,  0.0051,  ...,  0.0044,  0.0011,  0.0021]],
       device='cuda:0')

In [20]:
pmferror = PMFLoss(lam_u=0.05, lam_v=0.05)
optimizer = torch.optim.Adam([user_features, movie_features], lr=0.01)

bar = trange(num_epochs)
for epoch in bar:
    optimizer.zero_grad()
    loss = pmferror(user_bot_matrix, user_features, movie_features)
    loss.backward()
    optimizer.step()
    bar.set_postfix(loss=f'{loss:,.3f}')

  0%|          | 0/1000 [00:02<?, ?it/s]


RuntimeError: CUDA out of memory. Tried to allocate 1.04 GiB (GPU 0; 6.00 GiB total capacity; 4.69 GiB already allocated; 0 bytes free; 4.70 GiB reserved in total by PyTorch)